# Workflow example

In this example a single task is submitted to Argo via APIs using python lib.
The image that is used is the one created in the jupyther Notebook 2
It aim to create a whale with a balloon 

In [43]:
from argo.workflows.client import V1alpha1Api
from argo.workflows.config import load_kube_config

In [44]:
load_kube_config()

In [45]:
v1alpha1 = V1alpha1Api()

In [46]:
# Create a Workflow with a single task

from argo.workflows.client.models import *

container = V1Container(name="whale", image="localhost:32000/library/volume", command=["cowsay"], args=["hello world"])

metadata = V1ObjectMeta(generate_name="hello-world-")

template = V1alpha1Template(name="whalesay", container=container)

spec = V1alpha1WorkflowSpec(entrypoint="whalesay", templates=[template])

status = V1alpha1WorkflowStatus()
        
workflow = V1alpha1Workflow(
    api_version="argoproj.io/v1alpha1",
    kind="Workflow",
    metadata=metadata, 
    spec=spec,
    status=status)

In [47]:
# Get result: it contains workflow generated name in metadata.name
result = v1alpha1.create_namespaced_workflow('default', workflow)

In [48]:
wfs = v1alpha1.list_namespaced_workflows(namespace="default")

In [51]:
# The following fragment must be invoked more time, until the workflow is completed its execution

from kubernetes.client.rest import ApiException
from kubernetes import client, config

config.load_kube_config()

for job in wfs.items:
    if job.metadata.name == result.metadata.name:
        for pod_name in job.status.nodes.keys():
            try:
                api_instance = client.CoreV1Api()
                # Pod created by Argo has init containers, main container and wait container...
                api_response = api_instance.read_namespaced_pod_log(name=pod_name, container="main", namespace='default')
                print(api_response)
            except ApiException as e:
                print('Found exception in reading the logs')

 _____________ 
< hello world >
 ------------- 
    \
     \
      \     
                    ##        .            
              ## ## ##       ==            
           ## ## ## ##      ===            
       /""""""""""""""""___/ ===        
  ~~~ {~~ ~~~~ ~~~ ~~~~ ~~ ~ /  ===- ~~~   
       \______ o          __/            
        \    \        __/             
          \____\______/   

